<a href="https://colab.research.google.com/github/danigarod/danigarod/blob/main/Showcase_SMA_Crossover_Trading_Strategy_using_Python_backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required python libraries that do not come as default with Google Colab
!pip install yfinance
!pip install backtesting

     |████████████████████████████████| 63 kB 993 kB/s 
     |████████████████████████████████| 6.4 MB 23.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 175 kB 4.2 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173822 sha256=324c6ed9a11e0aee38a3496e6d95f9753ac4617547c20f12b8150de461f75

In [ ]:
# Import all pre-requisite python libraries to run this project
import yfinance as yf

import pandas as pd

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Create a dynamic input for the user to select a stock symbol (ticker)
ticker = input('Enter the ticker symbol you want to analyze: ')

# Import data from yahoo finance
ticker_data = yf.Ticker(ticker)
price_data = ticker_data.history(interval='1h', period='200d') # max amount of 1h data accessible via this api
price_data = price_data.iloc[:-1] # removing the last row of data due to a current bug in the backtesting library
price_data

Enter the ticker symbol you want to analyze: AMD


,Open,High,Low,Close,Volume,Dividends,Stock Splits
2021-04-07 09:30:00-04:00,81.129997,82.010002,80.360001,81.959999,9301964,0,0
2021-04-07 10:30:00-04:00,81.970001,82.309998,81.689003,81.901001,4016567,0,0
2021-04-07 11:30:00-04:00,81.910004,82.290001,81.533600,81.730003,4297509,0,0
2021-04-07 12:30:00-04:00,81.730003,81.985001,81.610001,81.870003,2986968,0,0
2021-04-07 13:30:00-04:00,81.870003,83.099998,81.807297,82.940002,5449983,0,0
...,...,...,...,...,...,...,...
2022-01-19 10:30:00-05:00,132.054993,132.369995,128.149994,128.850006,13402364,0,0
2022-01-19 11:30:00-05:00,128.850006,130.929993,128.020004,130.869995,9982285,0,0
2022-01-19 12:30:00-05:00,130.922104,131.440002,130.250107,130.630005,7318841,0,0
2022-01-19 13:30:00-05:00,130.630005,131.350006,129.539993,129.639999,6584879,0,0


In [ ]:
# Create a function that calcualtes the Simple Moving Average (SMA)
def Simple_Moving_Average(price, period):
  return pd.Series(price).rolling(period).mean()


class SMACrossOver(Strategy):
  interval_short_term = 50
  interval_long_term = 200

  def init(self):
    close = self.data.Close
    self.sma_short_term = self.I(Simple_Moving_Average, close, self.interval_short_term)
    self.sma_long_term = self.I(Simple_Moving_Average, close, self.interval_long_term)


  def next(self):
    price = self.data.Close[-1]

    if crossover (self.sma_short_term, self.sma_long_term):
      self.buy()
    elif crossover(self.sma_long_term, self.sma_short_term):
      if self.position:
        self.position.close()

backtest = Backtest(price_data, SMACrossOver, cash=10000, commission=0.002, exclusive_orders=True)
df = pd.DataFrame(backtest.run())
df


,0
Start,2021-04-07 09:30:00-04:00
End,2022-01-19 14:30:00-05:00
Duration,287 days 06:00:00
Exposure Time [%],57.9513
Equity Final [$],15276
Equity Peak [$],19499.8
Return [%],52.7603
Buy & Hold Return [%],57.7355
Return (Ann.) [%],70.5506
Volatility (Ann.) [%],59.6871


In [ ]:
# Plot the backtesting results so we can visualize them
backtest.plot()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning:

Superimposed OHLC plot matches the original plot. Skipping.



Row(id='1413', ...)

In [ ]:
# Define RSI just for quick compharison VS SMA buy/close signals
# define RSI calculation function
def calculate_RSI(close, lookback):
  delta = close.diff()
  up = []
  down = []
  for i in range(len(delta)):
    if delta[i] < 0:
      up.append(0)
      down.append(delta[i])
    else:
      up.append(delta[i])
      down.append(0)

  up_series = pd.Series(up)
  down_series = pd.Series(down).abs()

  up_ewm = up_series.ewm(com = lookback - 1, adjust = False).mean()
  down_ewm = down_series.ewm(com = lookback - 1, adjust = False).mean()

  relative_strength = up_ewm / down_ewm
  rsi = 100 - (100 / (1 + relative_strength))

  rsi_df = pd.DataFrame(rsi).rename(columns = {0: 'rsi'}).set_index(close.index)
  rsi_df = rsi_df.dropna()
  return rsi_df[3:]

price_data['RSI'] = calculate_RSI(price_data['Close'], 14)
price_data = price_data.dropna()

price_data = price_data.iloc[14:] # Skip first 14 days to have real values
price_data


,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI
2021-04-09 13:30:00-04:00,83.250000,83.589996,83.211197,83.309998,2485222,0,0,63.630799
2021-04-09 14:30:00-04:00,83.316902,83.431999,83.010002,83.187202,3224489,0,0,60.065817
2021-04-09 15:30:00-04:00,83.190002,83.309998,82.669998,82.760002,5820749,0,0,49.645021
2021-04-12 09:30:00-04:00,81.919998,82.108803,80.910004,81.550003,8150238,0,0,32.464912
2021-04-12 10:30:00-04:00,81.550003,81.949203,80.929802,81.709999,4942307,0,0,35.636653
...,...,...,...,...,...,...,...,...
2022-01-19 10:30:00-05:00,132.054993,132.369995,128.149994,128.850006,13402364,0,0,31.740425
2022-01-19 11:30:00-05:00,128.850006,130.929993,128.020004,130.869995,9982285,0,0,40.327686
2022-01-19 12:30:00-05:00,130.922104,131.440002,130.250107,130.630005,7318841,0,0,39.688851
2022-01-19 13:30:00-05:00,130.630005,131.350006,129.539993,129.639999,6584879,0,0,37.079412


In [ ]:
# Plot RSI
fig = go.Figure(make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.05))
fig.add_trace(
    go.Scatter(
      x=price_data.index,
      y=price_data['RSI'],
      name='RSI',
      marker_color='grey'
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=price_data.index,
        y=[70] * len(price_data.index),
        name='Overbought',
        marker_color='red',
        line=dict(dash='dot', width=5),
        showlegend=False
    ), row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=price_data.index,
        y=[30] * len(price_data.index),
        name='Oversold',
        marker_color='red',
        line=dict(dash='dot', width=5),
        showlegend=False
    ), row=1, col=1
)
fig.update_layout(
    title='RSI',
    title_font_size=26,
    title_x=0.5,
    yaxis_title='RSI',
    template='plotly_dark',
    height=800
)
fig.show()